In [92]:
import numpy as np
import pandas as pd
import plotly.express as px

from tasks.task1.main import generate_hilbert_matrix, generate_strong_tridiagonal_matrix

Реализовать решение СЛАУ двумя итерационными методами:методом простой итерации + методом Зейделя

In [68]:
def L_D_R_decomposition(A: np.array):
    D = np.tril(np.triu(A))
    L = np.tril(A) - D
    R = np.triu(A) - D
    return L, D, R


def solve_with_simple_iteration(A: np.array, b: np.array, epsilon=10 ** (-3)) -> np.array:
    n = A.shape[0]

    # checking on diagonal predominance
    diagonal_predominance = False
    non_diagonal_sum = 0
    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            non_diagonal_sum += abs(A[i, j])
    diagonal_min_abs = max(abs(element) for element in A.diagonal())
    if diagonal_min_abs >= non_diagonal_sum:
        diagonal_predominance = True

    # checking for positive definite matrices
    positive_definite = (np.allclose(A, A.T) and min(np.linalg.eig(A)[0]) >= 0)

    if diagonal_predominance:
        # print("Diagonal predominance\n")
        B = np.zeros((n, n))
        c = b.copy()
        for i in range(n):
            for j in range(n):
                if i != j:
                    B[i, j] = -(A[i, j] / A[i, i])
                else:
                    B[i, j] = 0
            c[i] = c[i] / A[i, i]
    elif positive_definite:
        # print("Positive definite\n")
        eigen_value = np.linalg.eig(A)[0]
        min_eig = min(eigen_value)
        max_eig = max(eigen_value)
        alfa = 2 / (min_eig + max_eig)
        B = np.eye(n) - alfa * A
        c = alfa * b
    else:
        raise RuntimeError("Matrix must have diagonal predominance or to be positive definite")

    B_norm = np.linalg.norm(B)
    max_eigen_value = max(np.linalg.eig(B)[0])
    assert max_eigen_value <= B_norm, f"Max eigen value {max_eigen_value} greater than B norm {B_norm}"

    x = c.copy()
    previous_x = None
    diff = None
    iteration_number = 0
    while previous_x is None or abs(diff) >= epsilon:
        previous_x = x
        x = B.dot(x) + c
        iteration_number += 1
        diff = np.linalg.norm(B) * np.linalg.norm(x - previous_x) / (1 - np.linalg.norm(B))
    return x, iteration_number


def solve_with_seidel_method(A: np.array, b: np.array, epsilon=10 ** (-3)) -> np.array:
    L, D, R = L_D_R_decomposition(A)
    B = (-1 * np.linalg.inv(D + L)) @ R
    c = np.linalg.inv(D + L) @ b

    x = c.copy()
    previous_x = None
    diff = None
    iteration_number = 0
    while previous_x is None or abs(diff) >= epsilon:
        previous_x = x
        x = B.dot(x) + c
        iteration_number += 1
        diff = np.linalg.norm(B) * np.linalg.norm(x - previous_x) / (1 - np.linalg.norm(B))
    return x, iteration_number

In [64]:
def solve_with_two_methods(A, b, info=False, epsilon=10 ** (-3)):
    expected = np.linalg.solve(A, b)
    if info:
        print(f"Expected: {expected}")

    actual_simple, iteration_number_simple = solve_with_simple_iteration(A, b, epsilon)
    if info:
        print(f"Discreapncy simple: {np.linalg.norm(actual_simple - expected)}")
        print(f"Iterations number simple: {iteration_number_simple}")

    actual_seidel, iteration_number_seidel = solve_with_seidel_method(A, b, epsilon)
    if info:
        print(f"Discreapncy sidel: {np.linalg.norm(actual_seidel - expected)}")
        print(f"Iterations number seidel: {iteration_number_seidel}")
    return expected, (actual_simple, iteration_number_simple), (actual_seidel, iteration_number_seidel)

In [65]:
A = np.array([[12, 1, 3],
              [1, 1, 0],
              [-1, -1, 1]], float)

b = np.array([1, 1, 1], float)

expected, (actual_simple, iteration_number_simple), (actual_seidel, iteration_number_seidel) = solve_with_two_methods(A,
                                                                                                                      b,
                                                                                                                      True)

Expected: [-0.54545455  1.54545455  2.        ]
Diagonal predominance

Discreapncy simple: 0.0002995838499435843
Iterations number simple: 19
Discreapncy sidel: 4.288389014461945e-05
Iterations number seidel: 4


In [66]:
A = np.array([[16, 5, 0],
              [5, 100, 4],
              [0, 4, 30]], float)

n = A.shape[0]
expected = np.ones(n)
b = A @ expected

expected, (actual_simple, iteration_number_simple), (actual_seidel, iteration_number_seidel) = solve_with_two_methods(A,
                                                                                                                      b,
                                                                                                                      True)

Expected: [1. 1. 1.]
Diagonal predominance

Discreapncy simple: 0.00015438559912934436
Iterations number simple: 4
Discreapncy sidel: 3.353512603311996e-06
Iterations number seidel: 3


In [77]:
def plot_iteration_steps_to_epsilon_degree(df):
    fig = px.line(df, x="epsilon degree", y="iteration steps", color="method")
    fig.show()

In [78]:
A = np.array([[16, 5, 0],
              [5, 100, 4],
              [0, 4, 30]], float)

n = A.shape[0]
expected = np.ones(n)
b = A @ expected

data = []
for i in range(-10, -1):
    epsilon = 10 ** i
    expected, (actual_simple, iteration_number_simple), (
    actual_seidel, iteration_number_seidel) = solve_with_two_methods(A, b, epsilon=epsilon)
    data.append([i, iteration_number_simple, "simple"])
    data.append([i, iteration_number_seidel, "seidel"])
df = pd.DataFrame(data, columns=["epsilon degree", "iteration steps", "method"])
plot_iteration_steps_to_epsilon_degree(df)

In [82]:
n = 2
A = generate_hilbert_matrix(n)
expected = np.ones(n)
b = A @ expected

data = []
for i in range(-10, -1):
    epsilon = 10 ** i
    expected, (actual_simple, iteration_number_simple), (
    actual_seidel, iteration_number_seidel) = solve_with_two_methods(A, b, epsilon=epsilon)
    data.append([i, iteration_number_simple, "simple"])
    data.append([i, iteration_number_seidel, "seidel"])
df = pd.DataFrame(data, columns=["epsilon degree", "iteration steps", "method"])
plot_iteration_steps_to_epsilon_degree(df)

In [83]:
n = 3
A = generate_hilbert_matrix(n)
expected = np.ones(n)
b = A @ expected

data = []
for i in range(-10, -1):
    epsilon = 10 ** i
    expected, (actual_simple, iteration_number_simple), (
    actual_seidel, iteration_number_seidel) = solve_with_two_methods(A, b, epsilon=epsilon)
    data.append([i, iteration_number_simple, "simple"])
    data.append([i, iteration_number_seidel, "seidel"])
df = pd.DataFrame(data, columns=["epsilon degree", "iteration steps", "method"])
plot_iteration_steps_to_epsilon_degree(df)

In [86]:
n = 4
A = generate_hilbert_matrix(n)
expected = np.ones(n)
b = A @ expected

data = []
for i in range(-3, -1):
    epsilon = 10 ** i
    expected, (actual_simple, iteration_number_simple), (
    actual_seidel, iteration_number_seidel) = solve_with_two_methods(A, b, epsilon=epsilon)
    data.append([i, iteration_number_simple, "simple"])
    data.append([i, iteration_number_seidel, "seidel"])
df = pd.DataFrame(data, columns=["epsilon degree", "iteration steps", "method"])
plot_iteration_steps_to_epsilon_degree(df)

In [95]:
n = 7
A = generate_hilbert_matrix(n)
expected = np.ones(n)
b = A @ expected

data = []
for i in range(-5, -1):
    epsilon = 10 ** i
    actual_seidel, iteration_number_seidel = solve_with_seidel_method(A, b, epsilon=epsilon)
    data.append([i, iteration_number_seidel, "seidel"])
df = pd.DataFrame(data, columns=["epsilon degree", "iteration steps", "method"])
plot_iteration_steps_to_epsilon_degree(df)

In [96]:
n = 7
A = generate_strong_tridiagonal_matrix(n)
expected = np.ones(n)
b = A @ expected

data = []
for i in range(-5, -1):
    epsilon = 10 ** i
    actual_seidel, iteration_number_seidel = solve_with_seidel_method(A, b, epsilon=epsilon)
    data.append([i, iteration_number_seidel, "seidel"])
df = pd.DataFrame(data, columns=["epsilon degree", "iteration steps", "method"])
plot_iteration_steps_to_epsilon_degree(df)